# LANL Earthquake Prediction Kaggle Competition
![signal](https://storage.googleapis.com/kaggle-competitions/kaggle/11000/logos/header.png)
by Michael Cascio<br>
<br>
[Research](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017GL076708) has shown that the statistical characteristics of acoustical data recorded during seismic events are indicative of fault zone shear stress and the frictional state of the fault system; this type of signal data can be used with [machine learnining (ML) to predict future fault failure](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017GL074677). The code below is my attempt at analyzing a continuous training segment of experimental seismic data, training a ML algorithm, and predicting future failure events.<br>
<br>
For this competition I decided to try to use the Yandex developed CatBoost open-source gradient boosting decision tree library. More on CatBoost can be found here: https://catboost.ai/

### Importing Required Packages:

In [65]:
import pandas as pd
import numpy as np
from scipy.stats import moment
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

### Loading Training Set:

In [66]:
train = pd.read_csv('train.csv', iterator=True, chunksize=150_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

### Generate Statistical Features of 150,000 Sample Signal Chunks
The training dataset is divided into chucks of 150,000 observations and a series of statistical properties are calculated for each. The CatBoostRegressor algorithim will find patterns in these statistical properties.

In [67]:
def gen_features(X):
    strain = []
    strain.append(X.mean())
    strain.append(X.std())
    strain.append(X.min())
    strain.append(X.max())
    strain.append(X.kurtosis())
    strain.append(X.skew())
    strain.append(np.quantile(X,0.01))
    strain.append(np.quantile(X,0.025))
    strain.append(np.quantile(X,0.05))
    strain.append(np.quantile(X,0.10))
    strain.append(np.quantile(X,0.15))
    strain.append(np.quantile(X,0.25))
    strain.append(np.quantile(X,0.30))
    strain.append(np.quantile(X,0.70))
    strain.append(np.quantile(X,0.75))
    strain.append(np.quantile(X,0.85))
    strain.append(np.quantile(X,0.90))
    strain.append(np.quantile(X,0.95))
    strain.append(np.quantile(X,0.075))
    strain.append(np.quantile(X,0.99))
    strain.append(np.abs(X).max())
    strain.append(np.abs(X).mean())
    strain.append(np.abs(X).std())
    strain.append(moment(X, moment=1))
    strain.append(moment(X, moment=2))
    strain.append(moment(X, moment=3))
    strain.append(moment(X, moment=10))
    return pd.Series(strain)

X = pd.DataFrame()
y = pd.Series()
for df in train:
    ch = gen_features(df['acoustic_data'])
    X = X.append(ch, ignore_index=True)
    y = y.append(pd.Series(df['time_to_failure'].values[-1]))

Shown below is a sample of the statistical characteristics calculated for the first five chucks of signal data.

In [68]:
X.head(5)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,4.884113,5.101106,-98.0,104.0,33.662481,-0.024061,-8.0,-4.0,-2.0,0.0,...,11.0,-1.0,18.0,104.0,5.576567,4.333325,0.0,26.021110,-3.193751,6.112766e+15
1,4.725767,6.588824,-154.0,181.0,98.758517,0.390561,-11.0,-5.0,-2.0,0.0,...,12.0,-1.0,21.0,181.0,5.734167,5.732777,0.0,43.412309,111.712897,1.286581e+18
2,4.906393,6.967397,-106.0,140.0,33.555211,0.217391,-15.0,-7.0,-3.0,0.0,...,13.0,-1.0,26.0,140.0,6.152647,5.895945,0.0,48.544298,73.526463,5.223283e+16
3,4.902240,6.922305,-199.0,197.0,116.548172,0.757278,-12.0,-5.0,-2.0,0.0,...,12.0,-1.0,22.0,199.0,5.933960,6.061214,0.0,47.917990,251.187842,5.775340e+18
4,4.908720,7.301110,-126.0,145.0,52.977905,0.064531,-15.0,-6.0,-2.0,0.0,...,12.0,-1.0,26.0,145.0,6.110587,6.329485,0.0,53.305855,25.114636,2.406886e+17


### Model Training and Evaluation
The following code implements the CatBoostRegressor algorithm. Mean Average Error is used as the loss function as it corresponds to how the competition's submissions are being evaluated

In [69]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.75)

cbr = CatBoostRegressor(iterations=10000, loss_function='MAE', boosting_type='Ordered')
cbr.fit(X_train, y_train, eval_set=(X_val, y_val),plot=True)
cbr.best_score_

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 5.8287228	test: 5.8376331	best: 5.8376331 (0)	total: 35ms	remaining: 5m 49s
1:	learn: 5.8141135	test: 5.8230716	best: 5.8230716 (1)	total: 62.5ms	remaining: 5m 12s
2:	learn: 5.7996115	test: 5.8084974	best: 5.8084974 (2)	total: 85.5ms	remaining: 4m 44s
3:	learn: 5.7851881	test: 5.7939444	best: 5.7939444 (3)	total: 111ms	remaining: 4m 38s
4:	learn: 5.7709047	test: 5.7794490	best: 5.7794490 (4)	total: 134ms	remaining: 4m 28s
5:	learn: 5.7566118	test: 5.7650054	best: 5.7650054 (5)	total: 153ms	remaining: 4m 14s
6:	learn: 5.7423698	test: 5.7506893	best: 5.7506893 (6)	total: 172ms	remaining: 4m 5s
7:	learn: 5.7282100	test: 5.7364575	best: 5.7364575 (7)	total: 190ms	remaining: 3m 57s
8:	learn: 5.7141957	test: 5.7223473	best: 5.7223473 (8)	total: 209ms	remaining: 3m 52s
9:	learn: 5.7002886	test: 5.7083916	best: 5.7083916 (9)	total: 229ms	remaining: 3m 48s
10:	learn: 5.6862964	test: 5.6942843	best: 5.6942843 (10)	total: 254ms	remaining: 3m 50s
11:	learn: 5.6726081	test: 5.6803600	best

96:	learn: 4.7167226	test: 4.7216544	best: 4.7216544 (96)	total: 1.91s	remaining: 3m 14s
97:	learn: 4.7075588	test: 4.7123547	best: 4.7123547 (97)	total: 1.93s	remaining: 3m 15s
98:	learn: 4.6985067	test: 4.7032549	best: 4.7032549 (98)	total: 1.96s	remaining: 3m 15s
99:	learn: 4.6891744	test: 4.6938549	best: 4.6938549 (99)	total: 1.98s	remaining: 3m 15s
100:	learn: 4.6803303	test: 4.6850565	best: 4.6850565 (100)	total: 2.02s	remaining: 3m 17s
101:	learn: 4.6712416	test: 4.6759859	best: 4.6759859 (101)	total: 2.04s	remaining: 3m 17s
102:	learn: 4.6623916	test: 4.6671418	best: 4.6671418 (102)	total: 2.06s	remaining: 3m 17s
103:	learn: 4.6534764	test: 4.6583301	best: 4.6583301 (103)	total: 2.07s	remaining: 3m 16s
104:	learn: 4.6445357	test: 4.6494154	best: 4.6494154 (104)	total: 2.08s	remaining: 3m 16s
105:	learn: 4.6356752	test: 4.6405780	best: 4.6405780 (105)	total: 2.1s	remaining: 3m 15s
106:	learn: 4.6269211	test: 4.6317410	best: 4.6317410 (106)	total: 2.11s	remaining: 3m 15s
107:	lea

195:	learn: 3.9221624	test: 3.9114890	best: 3.9114890 (195)	total: 3.55s	remaining: 2m 57s
196:	learn: 3.9150782	test: 3.9043006	best: 3.9043006 (196)	total: 3.56s	remaining: 2m 57s
197:	learn: 3.9079388	test: 3.8970236	best: 3.8970236 (197)	total: 3.58s	remaining: 2m 57s
198:	learn: 3.9008056	test: 3.8898676	best: 3.8898676 (198)	total: 3.59s	remaining: 2m 56s
199:	learn: 3.8937515	test: 3.8827031	best: 3.8827031 (199)	total: 3.61s	remaining: 2m 56s
200:	learn: 3.8865959	test: 3.8755261	best: 3.8755261 (200)	total: 3.62s	remaining: 2m 56s
201:	learn: 3.8794927	test: 3.8680956	best: 3.8680956 (201)	total: 3.64s	remaining: 2m 56s
202:	learn: 3.8728454	test: 3.8614014	best: 3.8614014 (202)	total: 3.65s	remaining: 2m 56s
203:	learn: 3.8662080	test: 3.8545731	best: 3.8545731 (203)	total: 3.67s	remaining: 2m 56s
204:	learn: 3.8591633	test: 3.8474350	best: 3.8474350 (204)	total: 3.69s	remaining: 2m 56s
205:	learn: 3.8520586	test: 3.8401866	best: 3.8401866 (205)	total: 3.7s	remaining: 2m 55s


296:	learn: 3.2837212	test: 3.2703209	best: 3.2703209 (296)	total: 5.21s	remaining: 2m 50s
297:	learn: 3.2779993	test: 3.2645471	best: 3.2645471 (297)	total: 5.22s	remaining: 2m 50s
298:	learn: 3.2724739	test: 3.2591456	best: 3.2591456 (298)	total: 5.24s	remaining: 2m 49s
299:	learn: 3.2668451	test: 3.2534686	best: 3.2534686 (299)	total: 5.25s	remaining: 2m 49s
300:	learn: 3.2612897	test: 3.2480465	best: 3.2480465 (300)	total: 5.26s	remaining: 2m 49s
301:	learn: 3.2557090	test: 3.2423466	best: 3.2423466 (301)	total: 5.27s	remaining: 2m 49s
302:	learn: 3.2503789	test: 3.2370135	best: 3.2370135 (302)	total: 5.28s	remaining: 2m 49s
303:	learn: 3.2450121	test: 3.2317694	best: 3.2317694 (303)	total: 5.3s	remaining: 2m 48s
304:	learn: 3.2394940	test: 3.2262618	best: 3.2262618 (304)	total: 5.31s	remaining: 2m 48s
305:	learn: 3.2340161	test: 3.2205948	best: 3.2205948 (305)	total: 5.33s	remaining: 2m 48s
306:	learn: 3.2286324	test: 3.2152288	best: 3.2152288 (306)	total: 5.34s	remaining: 2m 48s


...

9625:	learn: 1.5831473	test: 2.2283839	best: 2.2048667 (2589)	total: 3m 52s	remaining: 9.03s
9626:	learn: 1.5831413	test: 2.2283789	best: 2.2048667 (2589)	total: 3m 52s	remaining: 9.01s
9627:	learn: 1.5831042	test: 2.2283953	best: 2.2048667 (2589)	total: 3m 52s	remaining: 8.98s
9628:	learn: 1.5830696	test: 2.2284626	best: 2.2048667 (2589)	total: 3m 52s	remaining: 8.96s
9629:	learn: 1.5830277	test: 2.2285038	best: 2.2048667 (2589)	total: 3m 52s	remaining: 8.93s
9630:	learn: 1.5830212	test: 2.2284885	best: 2.2048667 (2589)	total: 3m 52s	remaining: 8.91s
9631:	learn: 1.5828851	test: 2.2285020	best: 2.2048667 (2589)	total: 3m 52s	remaining: 8.88s
9632:	learn: 1.5828746	test: 2.2284836	best: 2.2048667 (2589)	total: 3m 52s	remaining: 8.86s
9633:	learn: 1.5828532	test: 2.2284468	best: 2.2048667 (2589)	total: 3m 52s	remaining: 8.84s
9634:	learn: 1.5828157	test: 2.2284695	best: 2.2048667 (2589)	total: 3m 52s	remaining: 8.81s
9635:	learn: 1.5827650	test: 2.2285613	best: 2.2048667 (2589)	total: 3

9719:	learn: 1.5788380	test: 2.2292491	best: 2.2048667 (2589)	total: 3m 55s	remaining: 6.78s
9720:	learn: 1.5788028	test: 2.2292418	best: 2.2048667 (2589)	total: 3m 55s	remaining: 6.76s
9721:	learn: 1.5787845	test: 2.2292627	best: 2.2048667 (2589)	total: 3m 55s	remaining: 6.73s
9722:	learn: 1.5787041	test: 2.2292748	best: 2.2048667 (2589)	total: 3m 55s	remaining: 6.71s
9723:	learn: 1.5786959	test: 2.2292826	best: 2.2048667 (2589)	total: 3m 55s	remaining: 6.68s
9724:	learn: 1.5786544	test: 2.2292701	best: 2.2048667 (2589)	total: 3m 55s	remaining: 6.66s
9725:	learn: 1.5785092	test: 2.2293607	best: 2.2048667 (2589)	total: 3m 55s	remaining: 6.63s
9726:	learn: 1.5785027	test: 2.2293719	best: 2.2048667 (2589)	total: 3m 55s	remaining: 6.61s
9727:	learn: 1.5784139	test: 2.2293353	best: 2.2048667 (2589)	total: 3m 55s	remaining: 6.58s
9728:	learn: 1.5783662	test: 2.2293553	best: 2.2048667 (2589)	total: 3m 55s	remaining: 6.56s
9729:	learn: 1.5783053	test: 2.2293552	best: 2.2048667 (2589)	total: 3

9816:	learn: 1.5744897	test: 2.2295329	best: 2.2048667 (2589)	total: 3m 58s	remaining: 4.44s
9817:	learn: 1.5744718	test: 2.2295210	best: 2.2048667 (2589)	total: 3m 58s	remaining: 4.42s
9818:	learn: 1.5744378	test: 2.2295143	best: 2.2048667 (2589)	total: 3m 58s	remaining: 4.39s
9819:	learn: 1.5744130	test: 2.2295163	best: 2.2048667 (2589)	total: 3m 58s	remaining: 4.37s
9820:	learn: 1.5743461	test: 2.2296003	best: 2.2048667 (2589)	total: 3m 58s	remaining: 4.34s
9821:	learn: 1.5743084	test: 2.2296011	best: 2.2048667 (2589)	total: 3m 58s	remaining: 4.32s
9822:	learn: 1.5742797	test: 2.2295728	best: 2.2048667 (2589)	total: 3m 58s	remaining: 4.3s
9823:	learn: 1.5742658	test: 2.2295810	best: 2.2048667 (2589)	total: 3m 58s	remaining: 4.27s
9824:	learn: 1.5742470	test: 2.2295831	best: 2.2048667 (2589)	total: 3m 58s	remaining: 4.25s
9825:	learn: 1.5742275	test: 2.2295362	best: 2.2048667 (2589)	total: 3m 58s	remaining: 4.22s
9826:	learn: 1.5742061	test: 2.2295691	best: 2.2048667 (2589)	total: 3m

9910:	learn: 1.5703923	test: 2.2295530	best: 2.2048667 (2589)	total: 4m	remaining: 2.16s
9911:	learn: 1.5703224	test: 2.2295226	best: 2.2048667 (2589)	total: 4m	remaining: 2.13s
9912:	learn: 1.5703028	test: 2.2295194	best: 2.2048667 (2589)	total: 4m	remaining: 2.11s
9913:	learn: 1.5702795	test: 2.2294965	best: 2.2048667 (2589)	total: 4m	remaining: 2.09s
9914:	learn: 1.5701571	test: 2.2294643	best: 2.2048667 (2589)	total: 4m	remaining: 2.06s
9915:	learn: 1.5701442	test: 2.2294777	best: 2.2048667 (2589)	total: 4m	remaining: 2.04s
9916:	learn: 1.5701325	test: 2.2294701	best: 2.2048667 (2589)	total: 4m	remaining: 2.02s
9917:	learn: 1.5701076	test: 2.2294107	best: 2.2048667 (2589)	total: 4m 1s	remaining: 1.99s
9918:	learn: 1.5700371	test: 2.2294397	best: 2.2048667 (2589)	total: 4m 1s	remaining: 1.97s
9919:	learn: 1.5700101	test: 2.2294443	best: 2.2048667 (2589)	total: 4m 1s	remaining: 1.95s
9920:	learn: 1.5698902	test: 2.2294779	best: 2.2048667 (2589)	total: 4m 1s	remaining: 1.92s
9921:	lea

{'learn': {'MAE': 1.5661748318332385},
 'validation_0': {'MAE': 2.2048666776312498}}

### Prepping Test Data
The test data is a series of csv files each corresponding to acoustic signal data taken from a segment of a laboratory seismic test. The statistical characteristics listed above will be calculated for each of these segments and saved to a testing dataset. The trained CatBoostRegressor algorithm will be applied to this training dataset to predict time to earthquake.

In [ ]:
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')

In [71]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)
for seg_id in X_test.index:
    seg = pd.read_csv('~/git/Kaggle Earthquake/test/' + seg_id + '.csv')
    X_test.loc[seg_id] = gen_features(seg)
    
print(X_test)

                  0          1       2       3           4         5     6   \
seg_id                                                                        
seg_00030f  4.491780   4.893690   -75.0   115.0   28.837568  0.327908  -8.0   
seg_0012b5  4.171153   5.922839  -140.0   152.0   56.218955  0.295708 -12.0   
seg_00184e  4.610260   6.946990  -193.0   248.0  162.118284  0.428688 -11.0   
seg_003339  4.531473   4.114147   -93.0    85.0   41.241827  0.061889  -5.0   
seg_0042cc  4.128340   5.797164  -147.0   177.0   79.539708  0.073898 -10.0   
seg_004314  4.148607  24.782769  -675.0   671.0  159.651642  1.016361 -52.0   
seg_004cd2  4.113987   4.707150  -107.0   125.0   57.044041  0.081861  -7.0   
seg_004ee5  4.328380   5.964443  -120.0   120.0   35.504290  0.161541 -13.0   
seg_004f1f  4.000733   5.874469  -114.0   118.0   37.260235 -0.007195 -12.0   
seg_00648a  4.458800   8.926476  -258.0   281.0  133.899159 -0.123981 -17.0   
seg_006e4a  4.393327   5.568479   -73.0    91.0   20

### Making Predictions
Time to failure predictions are made on the test set and output to csv in the required format for submission to Kaggle. Personal results can be found here: https://www.kaggle.com/cascio/competitions.

In [72]:
submission['time_to_failure'] = cbr.predict(X_test)
submission.to_csv('submission.csv')